In [7]:
!pip install datasets
!pip install pandas

In [3]:
from datasets import load_dataset

dataset = load_dataset("Open-Orca/OpenOrca")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
!pip install argostranslate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 56.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895239 sha256=3b53ac8897608e69eb3f878305f0f39371002501f8eeb331a716670eadd0497a
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [5]:
import argostranslate.package
import argostranslate.translate

from_code = "en"
to_code = "bn"

# Download and install Argos Translate package
argostranslate.package.update_package_index()
available_packages = argostranslate.package.get_available_packages()
available_package = list(
    filter(
        lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
    )
)[0]
download_path = available_package.download()
argostranslate.package.install_from_path(download_path)

# Translate
installed_languages = argostranslate.translate.get_installed_languages()
from_lang = list(filter(
        lambda x: x.code == from_code,
        installed_languages))[0]
to_lang = list(filter(
        lambda x: x.code == to_code,
        installed_languages))[0]
translation = from_lang.get_translation(to_lang)
translatedText = translation.translate("Hello World!")
print(translatedText)
# '¡Hola Mundo!'

স্বাগত বিশ্ব!


In [ ]:
import pandas as pd
import argostranslate.package
import argostranslate.translate
from datasets import load_dataset
import time

# Load the original dataset
dataset = load_dataset("Open-Orca/OpenOrca")
df = pd.DataFrame(dataset['train'])

# Create a new DataFrame with only 'question' and 'response'
new_df = df[['question', 'response']].copy()

# Set up Argos Translate
from_code = "en"
to_code = "bn"
argostranslate.package.update_package_index()
available_packages = argostranslate.package.get_available_packages()
available_package = list(
    filter(
        lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
    )
)[0]
download_path = available_package.download()
argostranslate.package.install_from_path(download_path)

# Translate function
def translate_text(text):
    from_lang = list(filter(lambda x: x.code == from_code, argostranslate.translate.get_installed_languages()))[0]
    to_lang = list(filter(lambda x: x.code == to_code, argostranslate.translate.get_installed_languages()))[0]
    translation = from_lang.get_translation(to_lang)
    translated_text = translation.translate(text)
    return translated_text

# Translation loop
translated_pairs = 0
interval = 1800  # 30 minutes in seconds
start_time = time.time()

for index, row in new_df.iterrows():
    question = row['question']
    response = row['response']

    # Translate question and response
    translated_question = translate_text(question)
    translated_response = translate_text(response)

    # Update the DataFrame with translated text
    new_df.at[index, 'question'] = translated_question
    new_df.at[index, 'response'] = translated_response

    translated_pairs += 1

    # Check if 30 minutes have passed and save progress
    if time.time() - start_time > interval:
        new_df.to_csv('translated_data.csv', index=False)
        print(f"Translated pairs: {translated_pairs}")
        start_time = time.time()

# Save the final DataFrame
new_df.to_csv('translated_data_final.csv', index=False)
print("Translation completed. Total translated pairs:", translated_pairs)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
